In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]
import seaborn as sns

import netCDF4 as nc
import xarray as xr

from src.tools import MyNetCDF
from src.models import Fitter
from sklearn.linear_model import LinearRegression
import glob

In [14]:
ds = nc.Dataset('/Users/drigo/ITMO/_disser/surrogate/data/raw/sattelite/cr2smos_th_y2016.nc')
data = ds.variables['Thickness'][:]

In [2]:
thick = MyNetCDF(path='/Users/drigo/ITMO/_disser/surrogate/data/raw/sattelite/cr2smos_th_y2016.nc',
                variable='Thickness')
conc = MyNetCDF(path='/Users/drigo/ITMO/_disser/surrogate/data/raw/sattelite/conc_y2016.nc',
               variable='ice_conc')

/Users/drigo/ITMO/_disser/surrogate/src/tools.py:145: UserWarning: Mask was not set
  warnings.warn('Mask was not set')


In [23]:
t1=list(range(np.shape(conc.data)[0]))
t2=list(range(np.shape(conc.data)[1]))
t3=list(range(np.shape(conc.data)[2]))

len(np.indices(np.shape(conc.data))[0].ravel())

67165392

In [24]:
pd.DataFrame({'tstep':np.indices(np.shape(conc.data))[0].ravel(),
              'x':np.indices(np.shape(conc.data))[1].ravel(),
              'y':np.indices(np.shape(conc.data))[2].ravel(),
              'conc':conc.vector, 'thick':thick.vector})

,tstep,x,y,conc,thick
0,0,0,0,NaN,NaN
1,0,0,1,NaN,NaN
2,0,0,2,NaN,NaN
3,0,0,3,NaN,NaN
4,0,0,4,NaN,NaN
5,0,0,5,NaN,NaN
6,0,0,6,NaN,NaN
7,0,0,7,NaN,NaN
8,0,0,8,NaN,NaN
9,0,0,9,NaN,NaN


In [16]:
conc.dset.variables

OrderedDict([('longitude', <class 'netCDF4._netCDF4.Variable'>
              float32 longitude(y, x)
                  standard_name: longitude
                  long_name: longitude
                  units: degrees_east
                  _CoordinateAxisType: Lon
              unlimited dimensions: 
              current shape = (452, 406)
              filling off), ('latitude', <class 'netCDF4._netCDF4.Variable'>
              float32 latitude(y, x)
                  standard_name: latitude
                  long_name: latitude
                  units: degrees_north
                  _CoordinateAxisType: Lat
              unlimited dimensions: 
              current shape = (452, 406)
              filling off), ('time', <class 'netCDF4._netCDF4.Variable'>
              float64 time(time)
                  standard_name: time
                  long_name: reference time of product
                  units: seconds since 1978-1-1 00:00:00
                  calendar: standard
           

In [13]:
import netcdftime
netcdftime.num2date(conc.dset.variables['time'][:], units=conc.dset.variables['time'].units, calendar=u"gregorian")

array([datetime.datetime(2016, 1, 1, 12, 0),
       datetime.datetime(2016, 1, 2, 12, 0),
       datetime.datetime(2016, 1, 3, 12, 0),
       datetime.datetime(2016, 1, 4, 12, 0),
       datetime.datetime(2016, 1, 5, 12, 0),
       datetime.datetime(2016, 1, 6, 12, 0),
       datetime.datetime(2016, 1, 7, 12, 0),
       datetime.datetime(2016, 1, 8, 12, 0),
       datetime.datetime(2016, 1, 9, 12, 0),
       datetime.datetime(2016, 1, 10, 12, 0),
       datetime.datetime(2016, 1, 11, 12, 0),
       datetime.datetime(2016, 1, 12, 12, 0),
       datetime.datetime(2016, 1, 13, 12, 0),
       datetime.datetime(2016, 1, 14, 12, 0),
       datetime.datetime(2016, 1, 15, 12, 0),
       datetime.datetime(2016, 1, 16, 12, 0),
       datetime.datetime(2016, 1, 17, 12, 0),
       datetime.datetime(2016, 1, 18, 12, 0),
       datetime.datetime(2016, 1, 19, 12, 0),
       datetime.datetime(2016, 1, 20, 12, 0),
       datetime.datetime(2016, 1, 21, 12, 0),
       datetime.datetime(2016, 1, 22, 12, 0